##### Imports

In [10]:
from asvFormula.bayesianNetworks.bayesianNetwork import *
from asvFormula.bayesianNetworks import networkSamplesPath
from asvFormula.datasetManipulation import *
from asvFormula.exactASV import showASVandShapleyFor, ASV
import random 
from pgmpy.readwrite import BIFReader
from pgmpy.inference import VariableElimination
import numpy as np

#TODO: I should add classes and assign responsibilities to them. All of the functions are mixed and I have multiple variables flying. 

In [24]:
import importlib

import asvFormula.bayesianNetworks.bayesianNetwork

# Reload the module
importlib.reload(asvFormula.bayesianNetworks.bayesianNetwork)

<module 'asvFormula.bayesianNetworks.bayesianNetwork' from '/home/echu/tesis/pasantia-BICC/asvFormula/bayesianNetworks/bayesianNetwork.py'>

## Running ASV in Python

#### Train the decision tree model

In [12]:
random.seed(10)

cancerNetworkPath = networkSamplesPath + "/cancer.bif"
#cancerNetworkPath = networkSamplesPath + "/cancer_tree.bif"

BNmodel = BIFReader(cancerNetworkPath).get_model()

variableToPredict = "Pollution"
treeMaxDepth = 3
# Create a BNDatabaseGenerator object from the model
dataFromBN = datasetFromBayesianNetwork(BNmodel, 2000)

BNmodel.remove_node(variableToPredict) # We remove the variable to predict from the BN model so that we won't have this information when we are going to predict it
BNInference = VariableElimination(BNmodel)

featureColumns = list(dataFromBN.columns)
featureColumns.remove(variableToPredict)  
valuesPerFeature, encodedDataset = encodeCategoricalColumns(dataFromBN)
dtTreeClassifier = decisionTreeFromDataset(encodedDataset, variableToPredict , treeMaxDepth)

dtAsNetwork = obtainNetworkXTreeStructure(dtTreeClassifier, featureColumns)

  0%|          | 0/5 [00:00<?, ?it/s]

The model accuracy is : 0.8775


#### Compute the ASV vs the Shapley Value

In [13]:
asvCalc = ASV(BNmodel, dtTreeClassifier, BNInference, valuesPerFeature)

dataNoPrediction = encodedDataset.drop(variableToPredict, axis=1)
first_instance = dataNoPrediction.iloc[0]

showASVandShapleyFor(first_instance, featureColumns, dtTreeClassifier, asvCalc)


ASV values for the first instance:
ASV for Cancer: -2.220446049250313e-16
ASV for Xray: 0.0
ASV for Smoker: 0.0025725000000003106
ASV for Dyspnoea: 0.0
Sum of ASV values: 0.0025725000000000886
Shapley values for the first instance:
Feature: Cancer, Shapley Value: 0.0006280559424487835
Feature: Xray, Shapley Value: -0.005878832218506127
Feature: Smoker, Shapley Value: 0.014717880243693726
Feature: Dyspnoea, Shapley Value: 0.006701374293232981
Sum of Shapley values: 0.01616847826086936


#### Mean prediction of model

In [14]:
# Use the complete bayesian network to calculate the mean prediction
completeBNModel = BIFReader(cancerNetworkPath).get_model()
completeBNInference = VariableElimination(completeBNModel)

In [26]:
showMeanPredictionOfModel('high', variableToPredict, completeBNInference, valuesPerFeature, dtTreeClassifier)

Predicting the value high for the variable Pollution
Mean prediction value for the model: 0.09784322715719065
Estimated value for shap explainer: 0.103125
Probability in the bayesian network for the variable 0.1
